In [3]:
import CoolProp.CoolProp as CP
import numpy as np


In [4]:
def heat_transfer_coefficient(Pcon,Peva,CT_inlet,CT_outlet,AT,DT,ET,S2):
    print('Condenser Pressure (MPa): %0.3f'%(Pcon))
    Pcon=Pcon*1e6
    print('Evaporator Pressure (MPa): %0.3f'%Peva)
    Peva=Peva*1e6
    print('Condenser Inlet Temperature (C): %0.3f'%(CT_inlet))
    CT_inlet=CT_inlet+273
    print('Condenser Outlet Temperature (C): %0.3f'%CT_outlet)
    CT_outlet=CT_outlet+273
    print('Ambient Temperature (C): %0.3f'%(AT))
    AT=AT+273
    print('Dryer Temperature (C): %0.3f'%(DT))
    DT=DT+273
    print('Evaporator Temperature (C): %0.3f'%(ET))
    ET=ET+273
    print('Suction Temperature (C): %0.3f'%(S2))
    S2=S2+273
    #print('    P-H Data   ')
    hci=hg=CP.PropsSI('H','P',Pcon,'T',CT_inlet,'IsoButane')
    #print('hg (KJ/Kg): %0.3f'%(hg/1e3))
    
    hb=hh=CP.PropsSI('H','P',Pcon,'T',DT,'IsoButane')
    #print('hh(KJ/Kg): %0.3f'%(hh/1e3))
    #print('hb(KJ/Kg): %0.3f'%(hb/1e3))
    x=CP.PropsSI('Q','P',Peva,'H',hh,'IsoButane')
    
    x2=0.45*x    
    #print('Ideal Dryness factor (x): %0.3f'%x)
    #print('Modified Dryness factor (x): %0.3f'%x2)
    ha=CP.PropsSI('H','P',Peva,'Q',x2,'IsoButane')
    #print('ha(KJ/Kg): %0.3f'%(ha/1e3))
    hc=CP.PropsSI('H','P',Peva,'Q',1,'IsoButane')
    #print('hc(KJ/Kg): %0.3f'%(hc/1e3))
    hd=CP.PropsSI('H','P',Peva,'T',S2,'IsoButane')
    #print('hd(KJ/Kg): %0.3f'%(hd/1e3))
    #Condenser heat Radiation
    Hc=hg-hh
    #Evaporator heat absorption
    He=hc-ha
    Rce=Hc/He
    #print('Rce: %0.3f'%Rce)
    # Calculation condenser Kc
    Lc=23.22 
    G=1.122
    qe=(hd-hh)/1e3
    qc_back=qe*Rce
    #print('Condenser heat radiation (kj/kg) back: %0.3f'%qc_back)
    qc_forward=(hg-hh)/1e3
    #print('Condenser heat radiation (kj/kg) forward: %0.3f'%qc_forward)
    Qc=((qc_back*1e3)/3600)*G
    #print('Condenser Heat Radiation (w): %0.3f'%Qc)
    Kc=Qc/(Lc*(CT_outlet-AT))
    print('Kc: %0.3f'%Kc)
    
    # Colculation of Ke
    Qe=((qe*1e3)/3600)*G
    Eva_in=-29.9
    Eva_out=-28.8
    Eva=(Eva_in+Eva_out)/2
    PC=-1.6
    FC=-27.3
    Lfc=15.54
    Lpc=1.30
    Ke=Qe/((Lfc*(FC-Eva))+(Lpc*(PC-Eva)))
    print('Ke: %0.3f'%Ke)
    return Kc,Ke,Qe,Rce,Eva
            

In [6]:
Pcon=0.6
Peva=0.045
CT_inlet=89.3
CT_outlet=CP.PropsSI('T','P',Pcon*1e6,'Q',0,'IsoButane')
CT_outlet=CT_outlet-273
AT=38
dt=2
DT=CT_outlet-2
ET=-25
S2=37.2

In [7]:
Kc,Ke,Qe,Rce,Eva=heat_transfer_coefficient(Pcon,Peva,CT_inlet,CT_outlet,AT,DT,ET,S2)

Condenser Pressure (MPa): 0.600
Evaporator Pressure (MPa): 0.045
Condenser Inlet Temperature (C): 89.300
Condenser Outlet Temperature (C): 44.860
Ambient Temperature (C): 38.000
Dryer Temperature (C): 42.860
Evaporator Temperature (C): -25.000
Suction Temperature (C): 37.200
Kc: 0.816
Ke: 1.456


In [131]:
Ke

1.5199695306747303

In [132]:
V_FC=114
print('FC cabinet Volume (L): %0.2f'%V_FC)
V_PC=130
print('PC cabinet Volume (L): %0.2f'%V_PC)
Q_FC=96
print('FC cabinet Volume (w): %0.2f'%Q_FC)
Q_PC=53
print('PC cabinet Volume (w): %0.2f'%Q_PC)
Qe=Q_FC+Q_PC
Temp_FC=-27.3
Temp_PC=-1.6
print('FC cabinet Temperature (C): %0.2f'%Temp_FC)
print('PC cabinet Temperature (C): %0.2f'%Temp_PC)


FC cabinet Volume (L): 114.00
PC cabinet Volume (L): 130.00
FC cabinet Volume (w): 96.00
PC cabinet Volume (w): 53.00
FC cabinet Temperature (C): -27.30
PC cabinet Temperature (C): -1.60


In [133]:
def length_calculation(V_FC,V_PC,Q_FC,Q_PC,Temp_FC,Temp_PC,Kc,Ke,Qe,Rce,CT_outlet):
    Cfc=5.91
    Cpc=4.18
    Qd=Cfc*(V_FC**0.5)+Cpc*(V_PC**0.5)
    Q1=1   #Insulation
    Q2=1 #Gasket
    Q3=1 #Inner condenser
    Q4=1 #Rail tube
    #Qs=Q1+Q2+Q3+Q4
    Qs=Qe-Qd
    Qe=Qs+Qd
    print('Total load (w): %0.3f'%Qe)
    print('Static load (w): %0.3f '%Qs)
    print('Dynamic load (w): %0.3f'%Qd)
    
    Qc=Qe*Rce
    Lc_cal=(Qc)/(Kc*(CT_outlet-AT))
    print('Condenser Length (m): %0.3f'%Lc_cal)
    Lfc_cal=Q_FC/(Ke*(Temp_FC-Eva))
    print('Evaporator length for FC (m): %0.3f'%Lfc_cal)
    Lpc_cal=Q_PC/(Ke*(Temp_PC-Eva))
    print('Evaporator length for PC (m): %0.3f'%Lpc_cal)

In [134]:
length_calculation(V_FC,V_PC,Q_FC,Q_PC,Temp_FC,Temp_PC,Kc,Ke,Qe,Rce,CT_outlet)

Total load (w): 149.000
Static load (w): 38.239 
Dynamic load (w): 110.761
Condenser Length (m): 33.507
Evaporator length for FC (m): 30.809
Evaporator length for PC (m): 1.257
